In [1]:
import pandas as pd
import numpy as np

import os

os.chdir('results')

In [2]:
def get_metric(dataset, steps=30, eps=0.03, col='ACC'):
    result = pd.DataFrame()
    
    models = os.listdir(dataset)
    
    for model in models:
        try:
            data = pd.read_csv(dataset + '/' + model + f'/attack/fgsm_attackwithout_reg/aa_res_{dataset}_0.csv', index_col=0)
            data.columns = list(map(lambda x: x.strip(), data.columns) )

            result.loc[model, dataset] = data[(data['eps']==eps)&(data['n steps']==steps)][col].values[0]
        except:
            print(dataset, model)
    
    return result


def get_all(steps, eps, col='ACC'):
    
    folders = np.array(os.listdir())
    mask = list(map(lambda x: '.' not in x, folders))

    folders = folders[mask]
    
    result = pd.DataFrame()

    for dataset in folders:
        dataset = str(dataset)
        
        curr = get_metric(dataset, steps, eps, col)
        result[dataset] = curr
        
    return result

In [4]:
get_all(10, 0.03, 'HID')

Wafer RNNA
FordA RNNA
FreezerRegularTrain RNNA
FreezerRegularTrain LSTM


,ProximalPhalanxOutlineCorrect,Wafer,GunPointMaleVersusFemale,Strawberry,PowerCons,FordA,FreezerRegularTrain,Coffee
ResidualCNN,0.999038,0.998293,0.814168,1.000000,0.965016,1.000000,0.932317,0.982425
SeqS4,0.896747,0.760755,0.961766,0.881422,0.897946,0.701630,0.985324,0.999733
RNNA,0.879200,NaN,0.686518,0.989402,0.758814,NaN,NaN,0.997351
TS2VecClassifier,0.475629,0.968732,0.789800,0.950114,0.797593,0.788523,0.802953,0.976253
LSTM,0.968449,0.800530,0.787630,0.730099,0.398071,0.946581,NaN,NaN


In [5]:
get_all(10, 0.03, 'ACC')

Wafer RNNA
FordA RNNA
FreezerRegularTrain RNNA
FreezerRegularTrain LSTM


,ProximalPhalanxOutlineCorrect,Wafer,GunPointMaleVersusFemale,Strawberry,PowerCons,FordA,FreezerRegularTrain,Coffee
ResidualCNN,0.000000,0.006489,0.278481,0.000000,0.566667,0.003030,0.218246,0.0
SeqS4,0.006873,0.884977,0.496835,0.043243,0.838889,0.016667,0.054035,0.0
RNNA,0.082474,NaN,0.689873,0.121622,0.894444,NaN,NaN,0.0
TS2VecClassifier,0.000000,0.103342,0.509494,0.005405,0.244444,0.013636,0.287018,0.0
LSTM,0.010309,0.816029,0.705696,0.639865,0.700000,0.079167,NaN,NaN
